Complete the exercises below For **Assignment #4**.

Use **Markdown** cells for the non-code answers.

In this assignment we will work with the data underlying the *FiveThirtyEight* article ["Higher Rates Of Hate Crimes Are Tied To Income Inequality"](https://fivethirtyeight.com/features/higher-rates-of-hate-crimes-are-tied-to-income-inequality/).

Load the `tidymodels`, `readr`, and `moderndive` packages in the cell below.

In [28]:
library('tidymodels')
library('readr')
library('moderndive')



We can read the data from a **CSV file** at the following URL: [http://bit.ly/2ItxYg3](http://bit.ly/2ItxYg3).

Use the `read_csv` function to read the data into our R session. Call the new table `hate_crimes`.

In [29]:
hate_crimes = read_csv('http://bit.ly/2ItxYg3') |> drop_na()

hate_crimes|> head()

Rows: 51 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (5): state, median_house_inc, trump_support, unemployment, urbanization
dbl (4): share_pop_metro, hs, hate_crimes, income

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


state        median_house_inc share_pop_metro hs hate_crimes trump_support
1 New Mexico   low              0.69            83 0.295       low          
2 Maine        low              0.54            90 0.616       low          
3 New York     low              0.94            85 0.351       low          
4 Illinois     low              0.90            86 0.195       low          
5 Delaware     high             0.90            87 0.323       low          
6 Rhode Island high             1.00            85 0.095       low          
  unemployment urbanization income
1 high         low          46686 
2 low          low          51710 
3 low          high         54310 
4 high         high         54916 
5 low          high         57522 
6 high         high         58633

Next, let’s add the high-school degree variable into the mix by creating a scatterplot showing:

- Income on the y-axis (this is the `income` variable)
- Percent of adults 25 or older with a high school degree on the x-axis (this is the `hs` variable)
- The points colored by level of urbanization in a region (this is the variable `urbanization`)


**In addition, add a line of best fit (regression line) for each level of the variable urbanization (one for “low”, one for “high”).**

*Add the regression lines to the plot using the `geom_parallel_slopes` function from the `moderndive` package. This function will draw the regression lines based on fitting a regression model with parallel slopes (i.e., with no interaction between `hs` and `urbanization`).*

In [30]:
hate_crimes |> ggplot(aes(x = hs,y = income, color = urbanization))+
    geom_point()+
    geom_parallel_slopes(method='lm')




Warning message:
"`geom_parallel_slopes()` doesn't need a `method` argument ("lm" is used)."


❓Which regression line (high urbanization or low urbanization) appears to have the larger intercept?

**Answer:**

The high urbanization have higher intercept.



Now let’s create a second scatterplot using the same variables, but this time draw the regression lines using `geom_smooth(method = "lm")`, which will allow for separate, non-parallel slopes for each urbanization group. 

**Code your scatter plot in the cell below.**

In [31]:
hate_crimes |> ggplot(aes(x = hs,y = income, color = urbanization))+
    geom_point()+
    geom_smooth(method='lm')



`geom_smooth()` using formula = 'y ~ x'


❓Based on visually comparing the two models shown above, do you think it would be best to run a “parallel slopes” model (i.e. a model that estimates one shared slope for the two levels of urbanization), or a more complex “interaction model” (i.e. a model that estimates a separate slope for the two levels of urbanization)?

**Answer:**


I think it is best to use more complex model because the two levels of urbanization have different slope.


Fit the following two regression models that examine the relationship between household `income` (as response variable), and high-school education (`hs`) and `urbanization` as explanatory variables:

1. A parallel slopes model (i.e., no interaction between `hs` and `urbanization`). ❗️Save the data recipe and model under the variables `ps_rec` and `ps_mod`, respectively. 
1. A non-parallel slopes model (i.e., allow `hs` and `urbanization` to interact in your model). ❗️Save the data recipe and model under the variable: `nps_rec` and `nps_mod`, respectively.

**Code you your models in the cell below.**

In [32]:
# parallel slopes model

ps_rec = recipe(income ~ hs + urbanization, data = hate_crimes) |> 
    step_naomit(everything()) |>   # remove missing values
    step_dummy(urbanization) |>    # dummy encode the urbanization variable
    prep()                         # run the recipe on the training data provided

mod = linear_reg() |> set_engine('lm')

ps_mod = mod |> fit(income ~ ., juice(ps_rec))

ps_mod

parsnip model object


Call:
stats::lm(formula = income ~ ., data = data)

Coefficients:
     (Intercept)                hs  urbanization_low  
         -114372              1994             -7469  


In [38]:
# non-parallel slopes model

# ❗️ you can use most of the code above, just add step_interact(~ starts_with("urbanization"):hs) to the recipe and
#    switch the variable names...

nps_rec = recipe(income ~ hs + urbanization, data = hate_crimes) |> 
    step_naomit(everything()) |>   # remove missing values
    step_dummy(urbanization) |>    # dummy encode the urbanization variable
    step_interact(~ starts_with("urbanization"):hs) |>
    prep()                         # run the recipe on the training data provided

mod_np = linear_reg() |> set_engine('lm')

nps_mod = mod_np |> fit(income ~ ., juice(nps_rec))

nps_mod



parsnip model object


Call:
stats::lm(formula = income ~ ., data = data)

Coefficients:
          (Intercept)                     hs       urbanization_low  
             -95647.4                 1777.0               -37773.9  
urbanization_low_x_hs  
                350.5  


The following code creates a table of your model predictions over the training data. Calculate the [coefficient of determination](https://en.wikipedia.org/wiki/Coefficient_of_determination) (R<sup>2</sup>) for each model:

```r
rbind(
    augment(ps_mod, juice(ps_rec)) |> select(income, .pred, .resid) |> mutate(model = "parallel_slopes"),
    augment(nps_mod, juice(nps_rec)) |> select(income, .pred, .resid) |> mutate(model = "interaction")
)
```

1. Group rows by the `model` variable (use the `group_by` function).
1. Calculate the variance of income over the variance of your predictions for each model using the "grouped" data from the step above (use `summarize(r_squared = var(.pred) / var(income))`).

In [39]:

rbind(
    augment(ps_mod, juice(ps_rec)) |> select(income, .pred, .resid) |> mutate(model = "parallel_slopes"),
    augment(nps_mod, juice(nps_rec)) |> select(income, .pred, .resid) |> mutate(model = "interaction")
) |>
group_by(model) |>
summarize(r_squared = var(.pred) / var(income))


model           r_squared
1 interaction     0.6068800
2 parallel_slopes 0.6032128

🎶 Note: you can also use the `glance` function with a model as input to find the coefficient of determination.

In [40]:
glance(ps_mod)
glance(nps_mod)

r.squared adj.r.squared sigma    statistic p.value      df logLik   
value 0.6032128 0.5843182     6095.036 31.9251   3.712875e-09 2  -454.4852
      AIC      BIC      deviance   df.residual nobs
value 916.9705 924.1971 1560277443 42          45

r.squared adj.r.squared sigma    statistic p.value     df logLik   
value 0.60688   0.5781151     6140.345 21.09795  2.00388e-08 3  -454.2763
      AIC      BIC     deviance   df.residual nobs
value 918.5527 927.586 1545857222 41          45

❓Compare the adjusted proportion of variance accounted for in each model. Based on this comparison, which model do you prefer? Why? 

**Answer:**

❓Using your preferred model, based on your regression model parameters (and the data visualizations), is `income` greater in states that have lower or higher levels of `urbanization`? By how much?

**Hint:** use the `tidy` function with your model as input to access the parameters in a nice table.

**Answer:**

In [42]:

tidy(nps_mod)



term                  estimate    std.error  statistic  p.value     
1 (Intercept)           -95647.4274 38472.3854 -2.4861320 0.0170802887
2 hs                      1776.9938   446.2510  3.9820499 0.0002729535
3 urbanization_low      -37773.9105 49036.8457 -0.7703169 0.4455281058
4 urbanization_low_x_hs    350.4821   566.7254  0.6184338 0.5397101638

❓For every one percentage point increase of high-school educated adults in a state (`hs` variable), what is the associated average increase in `income`?

**Answer:**